## Vuex的使用

## Vue中的单向数据流

![](https://upload-images.jianshu.io/upload_images/7505161-9ada87a51805d828.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


```js
new Vue({
  // state
  data () {
    return {
      count: 0
    }
  },
  // view
  template: `
    <div>{{ count }}</div>
  `,
  // actions
  methods: {
    increment () {
      this.count++
    }
  }
})
```

- state，驱动应用的数据源；(data)
- view，以声明方式将 state 映射到视图；(template)
- actions，响应在 view 上的用户输入导致的状态变化。(methods)

应用遇到多个组件共享状态时，单向数据流的简洁性很容易被破坏：

- 多个视图依赖于同一状态。
- 来自不同视图的行为需要变更同一状态。

vue只承担`view`视图层的 内容.`Vuex`是一个数据框架.叫`状态管理模式`

![](https://upload-images.jianshu.io/upload_images/7505161-20404e01af8897c2.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


vuex指的是下面虚线的内容,`把公用的数据放在公共的存储空间里面去,某一个组件改变了公共的数据,其他组件就能感知到.`

你可以把公共的数据存储在`store`中.store中分为下面几个部分:
    - State(所有的数据都存储在State当中)
    - Actions(改变组件的数据,必须走Action这个流程:处理异步操作,或者复杂,批量的同步操作)
    - Mutations(放置对state一个一个同步的修改)
    
    组件要改变数据,必须先去调用Actions,Actions再去调用`Mutations`,最终改变公用数据的值.

有些时候,我们可以直接跳过`Actions`这一步,直接调用`Mutations`,修改公共数据的值.

### 操作方法
- 操作`Actions`的方法: `dispatch`
- 操作`Mutation`的方法: `commit`

    本质上是一个单项操作数据的流程.

## Vue中使用插件的形式

`Vue.use(vuex)`

### 使用Vuex实现数据共享

- target: 我们希望点击`City`页的的时候,首页的城市也跟着变.

- 分析: 需要City页的数据传递给首页.
- 难点: `City.vue`和`Home.vue`都是路由器,他们之间没有共同的父级组件.

要使用Vuex的组件是首页的'header'组件和`City`的`List`组件

**Change**

我们期望当热门城市发生单击事件发生改变的时候,当前城市也发生改变

```js
.button-wrapper(v-for="item of hot" :key="item.id" @click="handleCityClick(item.name)")
```

需要我们将`city`传入进去.往外触发一个事件.

```js
methods: {
    handleCityClick (name) {
      // 使用dispatch 来调用vuex中的actions
      // 需要我们将changeCity添加到store中
      this.$store.dispatch('changeCity', name)
    }
  },
```

**store.js中**

这样我们就在`store`中接收到数据.
```js
export default new Vuex.Store({
  state: {
    city: '北京'
  },
  actions: {
    changeCity (ctx, name) {
      // 下一步交给mutation去做,主要就是听过 ctx.commit
      ctx.commit('changeCity', name)
    }
  },
  mutations: {
    changeCity (state, name) {
      state.city = name
    }
  }
})

```

步骤: 组件调用`actions`,actions调用`mutation`

但是现在我们的逻辑很简单,可以直接使用`commit`来调用`mutation`方法

改为`this.$store.commit('changeCity', name)`,删除`actions:{}`即可

## vue-router的编程式导航

`this.$router.push('/')`

 ## Vuex的高级使用 & LocalStorage(本地存储)

HTML5中的新API `LocalStroge`,可以给我们提供类似Cookie的存储功能.

`LocalStorage.city = name`

如果浏览器处在隐身模式状态下,需要我们添加`try{}catch(e){}`来捕获异常.

## Vuex中的store的拆分

- index.js
- state.js
- mutations.js

## 使用Vuex进行代码优化

```js
import { mapState, mapMutations } from 'vuex'

computed: {
    // 将 city这个公用数据,映射到名字叫做city的计算属性当中
    ...mapState(['city'])
  }

methods: {
    handleCityClick (name) {
      // 使用dispatch 来调用vuex中的actions
      // 需要我们将changeCity添加到store中
      // this.$store.commit('changeCity', name)
      this.changeCity(name)
      this.$router.push('./')
    },
    // 现在有一个mutation叫changeCity,现在将changeCity映射到这个组件当中.
    ...mapMutations(['changeCity'])
  },

```

直接使用`this.city`,`this.changeCity(name)`就可以了.

**getters的使用**

getters类似计算属性.

```js
import { mapGetters} from 'vuex'
computed:{
...mapGetters(['doubleCity'])
}
```

在`store.js`中

```js
 getters: {
    doubleCity (state) {
      return state.city + '' + state.city
    }
  }
```

**module的使用**

将 store 分割成模块（module）

```js
const moduleA = {
  state: { ... },
  mutations: { ... },
  actions: { ... },
  getters: { ... }
}

const moduleB = {
  state: { ... },
  mutations: { ... },
  actions: { ... }
}

const store = new Vuex.Store({
  modules: {
    a: moduleA,
    b: moduleB
  }
})

store.state.a // -> moduleA 的状态
store.state.b // -> moduleB 的状态
```